In [16]:
import cv2
import mediapipe as mp
import pandas as pd
import re
from moviepy.editor import *
import time

import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_holistic = mp.solutions.holistic

df = pd.DataFrame()
frames = []

In [17]:
import os
import copy
path = os.getcwd()

In [18]:
import math
import matplotlib.pyplot as plt

def normalize_coordinates(coordinates):
    max_x = max(coord[0] for coord in coordinates)
    max_y = max(coord[1] for coord in coordinates)
    
    normalized_coordinates = []
    for x, y in coordinates:
        normalized_x = x / max_x
        normalized_y = y / max_y
        normalized_coordinates.append([normalized_x, normalized_y])
    
    return normalized_coordinates

def visualize_data(normalized_coordinates):
    # Unzip normalized coordinates for plotting
    normalized_x, normalized_y = zip(*normalized_coordinates)
    
    # Plot the normalized coordinates
    plt.figure(figsize=(8, 6))
    plt.scatter(normalized_x, normalized_y, color='blue', label='Normalized Coordinates')
    plt.xlabel('Normalized X')
    plt.xlabel('Normalized X')
    plt.ylabel('Normalized Y')
    plt.title('Normalized Coordinates Plot')
    plt.legend()
    plt.grid(True)
    plt.show()


In [19]:
def generate_centered_image(detected_pixels):
    normalized_coordinates = normalize_coordinates(detected_pixels)
    # visualize_data(normalized_coordinates)

    return normalized_coordinates

In [20]:
def extract_video(image, target, sequence_id):
    global frames
    detected_pixels = []

    # Load an image from file
    image = cv2.imread(image)
    # Check if the image was loaded successfully
    if image is not None:
        # Convert the BGR frame to RGB
        rgb_frame = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Process the frame with the hand tracking model
        results = hands.process(rgb_frame)
         
        if results.multi_hand_landmarks:
            row_data = {
                "sequence_id": sequence_id,
                "target": target
            }
            hand_side = "Left"
            detected_hands = []
            for idx, landmarks in enumerate(results.multi_hand_landmarks):
                # Iterate through detected hand landmarks
                for landmark_idx, landmark in enumerate(landmarks.landmark):
                    x, y = landmark.x * image.shape[1], landmark.y * image.shape[0]
                    detected_pixels.append([x, y])

                    # Draw circles on the frame
                    cv2.circle(image, (int(x), int(y)), 5, (0, 255, 0), -1)
                    cv2.putText(image, hand_side, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                break  
            normalized_coordinates_relative = generate_centered_image(detected_pixels)
            if (len(normalized_coordinates_relative) > 21):
                print(target, sequence_id, normalized_coordinates_relative)
            for i in range(len(normalized_coordinates_relative)):
                x, y = normalized_coordinates_relative[i]
                row_data[f'x_{hand_side}_hand_{i}'] =  x
                row_data[f'y_{hand_side}_hand_{i}'] =  y

            frames.append(row_data)
        
        # cv2.imshow('Original', image)
        
    else:
        print("Failed to load image.")

    # cv2.waitKey(0)
    cv2.destroyAllWindows()

In [21]:
import glob, os
import time
os.chdir("D:\Documents\Trabajo\megaproyecto_model\data/asl_letters")

sequence_id = 0
for file in glob.glob("*.png"):
    sequence_id += 1
    name = " ".join(file.split(" ")[:-1])

    input_path = f'D:\Documents\Trabajo\megaproyecto_model\data/asl_letters/{file}'


    extract_video(input_path, name, sequence_id)

In [22]:
df = pd.DataFrame(frames)

In [23]:
df['sequence_id'] = df['sequence_id'].astype(int)

In [24]:
len(df)

152

In [25]:
df.head()

,sequence_id,target,x_Left_hand_0,y_Left_hand_0,x_Left_hand_1,y_Left_hand_1,x_Left_hand_2,y_Left_hand_2,x_Left_hand_3,y_Left_hand_3,...,x_Left_hand_16,y_Left_hand_16,x_Left_hand_17,y_Left_hand_17,x_Left_hand_18,y_Left_hand_18,x_Left_hand_19,y_Left_hand_19,x_Left_hand_20,y_Left_hand_20
0,1,a,0.530948,1.0,0.715785,0.944666,0.868372,0.737896,0.921069,0.546306,...,0.544049,0.777359,0.379488,0.578000,0.403089,0.486595,0.425640,0.630759,0.439509,0.707957
1,2,a,0.598715,1.0,0.817269,0.787746,0.907771,0.568508,0.940256,0.383617,...,0.604673,0.687549,0.428759,0.565205,0.488588,0.429457,0.528684,0.529194,0.526195,0.620994
2,3,a,0.699040,1.0,0.855275,0.846713,0.927487,0.655646,0.931123,0.462656,...,0.683310,0.719400,0.607122,0.609145,0.574001,0.512297,0.590989,0.615578,0.620398,0.693777
3,4,a,0.634597,1.0,0.810858,0.903572,0.907139,0.784950,0.954674,0.676814,...,0.645475,0.802731,0.455929,0.711337,0.475522,0.609587,0.522922,0.668138,0.547249,0.736916
4,5,a,0.623369,1.0,0.788632,0.951898,0.908468,0.803502,0.940904,0.680150,...,0.618199,0.885746,0.434740,0.745519,0.459325,0.661971,0.500133,0.754159,0.523836,0.827396


In [26]:
dir_data = "D:\Documents\Trabajo\megaproyecto_model/data" + "/data_letters.csv"
df.to_csv(dir_data, index=False)

In [27]:
len(df.columns)

44

In [28]:
validation_path = "D:\Documents\Trabajo\megaproyecto_model/data" + "/validation_letters.csv"

import random
grouped = df.groupby('target')
selected_validation_ids = []
for group_name, group_indices in grouped.groups.items():
    ids = list(set(df.loc[group_indices]['sequence_id'].values))
    if (len(ids) < 0):
        raise ("Error")
    selected_validation_ids.append(random.choice(ids))

validation_df = df[df['sequence_id'].isin(selected_validation_ids)][["sequence_id", "target"]]
validation_df.to_csv(validation_path, index=False)

In [29]:
train_path = "D:\Documents\Trabajo\megaproyecto_model/data" + "/train_letters.csv"
train_df = df[~df['sequence_id'].isin(selected_validation_ids)][["sequence_id", "target"]]
train_df.to_csv(train_path, index=False)